# Mode Explorer Sonification

## 1. Dependencies:
* numpy
* cython
* matplotlib

## 2. Description:
Based on Particle Trajectory Sonification. Sonify mode of high dimensional numerical data. Interact with GUI window.

In [3]:
# from __future__ import division
%load_ext Cython
# %pylab qt4
# %pylab
import matplotlib.pyplot as plt
import numpy as np
import math, time, random, threading
import pyo

# If no pop up run below
# %pylab qt4

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython

WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter.
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



In [4]:
s = pyo.Server(sr=11025, nchnls=1, buffersize=512)  # pyo server
s.boot()
fifo = pyo.FIFOPlayer()
out = pyo.Biquad(fifo, freq=5000, q=1, type=0).out()  # BPF
s.start()
# play data by pushing it into the fifo pipe, e.g.
# fifo.put(vector.astype(np.float32))

## 3. Load Demo Data

In [18]:
class DataGen():
    """create test data"""
    def stddata(self, data, dim):
        """Normalize data"""
        for i in range(dim):
            data[:, i] = (data[:, i] - np.mean(data[:, i])) / np.std(data[:, 1])
            data[:, i] = data[:, i] / np.max(np.absolute(data[:, i]))/2
        return data
    
    def datagen(self, dim, c, sigma=0.2, minnr=50, maxnr=200):
        """Generate data set based of dimension and num_of_cluster."""
        for i in range(c):
            nr = random.randrange(minnr, maxnr, 1)
            meanvec = np.random.rand(dim)
            covmat = sigma ** 2 * np.cov(np.random.rand(dim, dim))
            dtmp = np.random.multivariate_normal(meanvec, covmat, nr)
            if (i == 0): result = dtmp.copy()
            else: result = np.vstack((result, dtmp.copy()))
        return self.stddata(result, dim)
    
def nneighbor(data, pos):
    """find index of nearest row vector in data to vector pos."""
    N, dim = np.shape(data)
    if(np.shape(pos)[0]<>dim): 
        raise AttributeError("Wrong dimension of pos")
    sqdists = sum(np.square(data - np.tile(pos, (N, 1))), 1)
    return np.argmin(sqdists)

In [19]:
# Generate fake data
genNrmin = 60
genNrmax = 250
dim = 5 # dimension
nc = 3  # clusters
data = DataGen().datagen(dim, nc, sigma=0.4, minnr=genNrmin, maxnr=genNrmax) * 2
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(data[:,0], data[:,1], '.')
N, dim  = np.shape(data)
s = np.std(data)
data_dev = np.std(data)
sigmaBuf = 0.0
data_dev = np.std(data)
print N, dim

487 5


## Trajectory
This is the same code for the Particle Trajectory Sonification 

In [20]:
%%cython -2
cimport numpy as np
import numpy as np
from libc.stdlib cimport rand, malloc, free
from libc.math cimport exp

def potential(np.ndarray[np.float64_t, ndim=2] data,\
              np.ndarray[np.float64_t, ndim=1] pos,\
              double sigma=0.2):
    """Potential energy function. 
    
    Args:
      data (float64): data 
      
      pos (float64): current position of the particle
      
      sigma (double): sigma attribute

    Returns:
      potential double : potential energy of the particle
    """
    cdef int N, dim, j, i
    cdef double potential, distsq, h
    cdef double minusOneOverTwoSigmaSquared = -0.5/(sigma*sigma)
    N, dim = data.shape[0], data.shape[1]
    for j in range(N):
        distsq = 0
        for i in range(dim): 
            h = pos[i] - data[j, i]
            distsq += h*h
        potential += -exp(minusOneOverTwoSigmaSquared * distsq)
    return potential

# trj, sig, lastpos, lastvel = PTSM(data, initialpos, initialvel, sigma, mass, r, dt, nrSteps)
def PTSM(np.ndarray[np.float64_t, ndim=2] data,\
         np.ndarray[np.float64_t, ndim=1] initialpos,\
         np.ndarray[np.float64_t, ndim=1] initialvel,\
         double sigma=0.25, double mass=1,\
         double r=0.99, double dt=0.01, int nrSteps=1000):
    """Particle trajectory, calculate the potential energy 
    at each position in the data space
    and figure where to go next. """
    cdef int N, dim, i, j, step
    N, dim = data.shape[0], data.shape[1]
    cdef double sigma2, m, dist_sq_sum, vel_sq_sum, dt_over_m, hlp
    cdef double *force    = <double *>malloc(dim * sizeof(double))
    cdef double *velocity = <double *>malloc(dim * sizeof(double))
    cdef double *position = <double *>malloc(dim * sizeof(double))
    cdef double *tmp      = <double *>malloc(dim * sizeof(double))  
    trj = np.zeros(nrSteps*dim, dtype=np.float64)
    sig = np.zeros(nrSteps,     dtype=np.float64)
    sigma2    = sigma * sigma
    m         = mass / sigma2  # division by sigma for sigma-independent pitch
    dt_over_m = dt / m
    for i in range(dim):
        position[i] = initialpos[i]
        velocity[i] = initialvel[i]
    for step in range(nrSteps): 
        for i in range(dim): force[i]=0
        for j in range(N):
            dist_sq_sum = 0
            for i in range(dim):
                tmp[i] = position[i] - data[j,i]
                dist_sq_sum += tmp[i] * tmp[i]
            hlp = exp(-dist_sq_sum/sigma2)/sigma2
            for i in range(dim):
                force[i] += -tmp[i]*hlp
        # numerical integration => update pos and vel 
        vel_sq_sum = 0
        for i in range(dim):
            velocity[i] =  r * velocity[i] + force[i] * dt_over_m
            position[i] += dt * velocity[i]
            vel_sq_sum  += velocity[i] * velocity[i] 
        # store  
        sig[step] = vel_sq_sum
        offset    = step*dim
        for i in range(dim): 
            trj[offset+i] = position[i]
    # prepare return values
    trj     = np.reshape(trj, (-1, dim)) # correct array shape 
    lastpos = np.zeros(dim, dtype=np.float64)
    lastvel = np.zeros(dim, dtype=np.float64)
    for i in range(dim): 
        lastpos[i]=position[i]; 
        lastvel[i]=velocity[i]
    # free memory
    free(force); free(velocity); free(position); free(tmp)
    return trj, sig, lastpos, lastvel

## Mode Explorer Sonification
The following code moves on click quickly towards the mode using the PTSM with high damping to perform the gradient descent, and then initializes sonification with the tiny oscillations around the mode...

In [36]:
fig, axarr = plt.subplots(1,2, gridspec_kw = {'width_ratios':[5,3]})
mngr = plt.get_current_fig_manager()
try:
    stopevent.set() 
except NameError:
    pass
ix=0; iy=1
sigma   = 0.5 * s
mass    = 1.0 
r       = 0.999
dt      = 0.1
nrSteps = 256
sigmaBuf = sigma

dsplt, = axarr[0].plot(data[:,ix], data[:,iy], "go", markersize=3)
trplt, = axarr[0].plot([], "r-", lw=5)
stplt, = axarr[0].plot([], "bo", markersize = 2)
sgplt, = axarr[1].plot([0,200],[0,2]  , "b-")
# sigplt, = axarr[2].plot(0.5, sigma, 'bo', markersize=16)
# axarr[2].axis([0, 1., 0, 2. * s])
plt.show(block=False)
fig.canvas.draw()
lastpos = np.zeros(dim)
lastvel = np.zeros(dim)
maxamp = 1.
lock = False

def on_click(event): 
    global lock; lock = True
    
def on_release(event): 
    global lock; lock = False
    
def on_motion(event):
    global lock, sigma, mass, r, dt, nrSteps, sig, trj, lastpos, lastvel, sigmaBuf
    global lastTime, waitTime, maxamp
    if (lock):
        sigma = sigmaBuf
        pos2d = [event.xdata, event.ydata]
        lastposmf = data[nneighbor(data[:,[ix,iy]], pos2d),:]
        modeFound = False
        mfcnt = 0
        while(not modeFound and mfcnt<100):
            mfcnt +=1
            trjjunk, sigjunk, lastposmf, velJunk = PTSM(data, lastposmf, np.zeros(dim), sigma, mass, 0, 0.1, 100)
            if(np.linalg.norm(velJunk)<0.00001): modeFound=True
        modepot = potential(data, lastposmf, sigma)
        # print mfcnt, modepot
        if(mfcnt<100):
            lastvel = np.random.rand(dim) * 0.03 * sigma
            lastpos = lastposmf
            trjjj, sigjj, lastposjj, lastveljj = PTSM(data, lastpos, lastvel, sigma, mass, 1, dt, 4*nrSteps)
            maxamp = max(sigjj)
        else:
            maxamp = 10000
    time.sleep(0.02)

def handle_close(evt):
    stopevent.set() 

fig.canvas.mpl_connect('close_event', handle_close)
cidclk = fig.canvas.mpl_connect('button_press_event', on_click)
cidrel = fig.canvas.mpl_connect('button_release_event', on_release)
cidmov = fig.canvas.mpl_connect('motion_notify_event', on_motion)

def proc(fifo, stopevent):
    global pos, r, sigma, mass, dt, nrSteps, lastpos, lastvel, trj, sig, ix, iy, axarr , alpha
    global maxamp
    sent = 0
    srate = 11025.0
    t0 = time.time()
    while not stopevent.wait(0):
        nrSteps = 100 + int(random.random()*200)
        sent = sent + nrSteps
        trj, sig, lastpos, lastvel = PTSM(data, lastpos, lastvel, sigma, mass, r, dt, nrSteps)
        if(max(sig) > 5 * maxamp): 
            sig = sig * 0 
        fifo.put(sig.astype(np.float32) / maxamp * 0.2)
        dti = sent / srate - (time.time() - t0 - 0.15)
        if(dti>0.12):   
            trplt.set_data(trj[:,ix], trj[:,iy])
            sgplt.set_data(np.arange(np.shape(sig)[0]), sig/max(abs(sig)))
            sigplt.set_data(0.5, sigma)
            axarr[0].draw_artist(axarr[0].patch)
            axarr[0].draw_artist(dsplt)
            axarr[0].draw_artist(trplt)
#             axarr[1].draw_artist(axarr[1].patch)
#             axarr[1].draw_artist(sgplt)
            fig.canvas.draw()
            fig.canvas.flush_events()
        dti = sent/srate - (time.time()-t0-0.03)
        if(dti>0): time.sleep(dti)  
    print "Mode Explorer Cleared."
    
    
stopevent = threading.Event()
producer = threading.Thread(name="Compute audio signal", target=proc, args=[fifo, stopevent])
producer.start()

In [37]:
# Manually update sigmaBuf
sigmaBuf = 0.2 * s

Mode Explorer Cleared.


## With Long Decay 
This is similar to the Particle Trajectory Sonification except sound is only played when the particle reaches the mode.

In [34]:
fig, axarr = plt.subplots(1,2, gridspec_kw = {'width_ratios':[5,3]})
mngr = plt.get_current_fig_manager()
mngr.window.setGeometry(0, 150, 1000, 700)

try:
    stopevent.set() 
except NameError:
    pass

ix=0; iy=1
sigma   = 0.6*s
mass    = 1.0 
r       = 0.9995
dt      = 0.1
nrSteps = 256
sigmaBuf = sigma
dsplt, = axarr[0].plot(data[:,ix], data[:,iy], "go", markersize=3)
trplt, = axarr[0].plot([], "r-", lw=1)
stplt, = axarr[0].plot([], "bo", markersize = 2)
sgplt, = axarr[1].plot([0,200],[0,2]  , "b-")

plt.show(block=False)
fig.canvas.draw()
lastpos = np.zeros(dim)
lastvel = np.zeros(dim)
maxamp = 1.

def on_click(event):
    """Get position info on click"""
    global lastpos
    pos2d = [event.xdata, event.ydata]
    lastpos = data[nneighbor(data[:,[ix,iy]], pos2d),:]
def handle_close(evt):
    stopevent.set() 

fig.canvas.mpl_connect('close_event', handle_close)
cidclk = fig.canvas.mpl_connect('button_press_event', on_click)


def proc(fifo, stopevent):
    """Main audio/visual processing function"""
    global pos, r, sigma, mass, dt, nrSteps, lastpos, lastvel, trj, sig, ix, iy, axarr , alpha
    global maxamp
    sent = 0
    srate = 11025.0
    t0 = time.time()
    while not stopevent.wait(0):
        nrSteps = 100 + int(random.random()*200)
        sent = sent + nrSteps
        sigmaBuf = sigma
        # print sigma
        trj, sig, lastpos, lastvel = PTSM(data, lastpos, lastvel, sigma, mass, r, dt, nrSteps)
        if(max(sig)>5*maxamp): sig = sig * 0 
        fifo.put(sig.astype(np.float32)/maxamp*0.2)
        dti = sent/srate - (time.time()-t0-0.15)
        if(dti>0.12):   
            trplt.set_data(trj[:,ix], trj[:,iy])
            sgplt.set_data(np.arange(np.shape(sig)[0]), sig / max(abs(sig)))
            sigplt.set_data(0.5, sigma)
            axarr[0].draw_artist(axarr[0].patch)
            axarr[0].draw_artist(dsplt)
            axarr[0].draw_artist(trplt)
            fig.canvas.update()
            fig.canvas.flush_events()
        dti = sent/srate - (time.time()-t0-0.03)
        if(dti>0):
            time.sleep(dti)  
    print "Mode Explorer Cleared."
    

stopevent = threading.Event()
producer = threading.Thread(name="Compute audio signal", target=proc, args=[fifo, stopevent])
producer.start()

[-0.4679321335591229, 0.7712402179774192]


In [35]:
# Play around with parameters
sigma   = 0.3*data_dev
r       = 0.9995

[None, None]


Traceback (most recent call last):
  File "/Users/jiajunyang/anaconda3/envs/mode_explorer/lib/python2.7/site-packages/matplotlib/cbook/__init__.py", line 387, in process
    proxy(*args, **kwargs)
  File "/Users/jiajunyang/anaconda3/envs/mode_explorer/lib/python2.7/site-packages/matplotlib/cbook/__init__.py", line 227, in __call__
    return mtd(*args, **kwargs)
  File "<ipython-input-34-9eb8a14f1815>", line 33, in on_click
    lastpos = data[nneighbor(data[:,[ix,iy]], pos2d),:]
  File "<ipython-input-18-a73f97c286bb>", line 26, in nneighbor
    sqdists = sum(np.square(data - np.tile(pos, (N, 1))), 1)
TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'


[-0.56381360288694, 0.8632788313019562]
[-0.42478547236160513, 0.687967186874267]
[0.1984440782692063, -0.13161475082517948]
[0.19365000480281536, -0.6005733996692476]
[-0.7076358068786658, 0.5301867068893469]
Mode Explorer Cleared.
